# https://www.youtube.com/watch?v=01sAkU_NvOY

The volume control method was used implemented to trigger the writing mode.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
import tkinter as tk
from collections import deque
import matplotlib.pyplot as plt
from keras.models import load_model
from collections import deque

model = load_model('Assesment3AAImodelJohnsonPaku.h5')

cap = cv2.VideoCapture(0)
wcam,hcam =1200, 480
cap.set(3,wcam)
cap.set(4,hcam)
red_low = np.array([0, 0, 255])
red_high = np.array([0, 0, 255])
test_image = 0

def image_conversion(test):
    test = cv2.resize(test, (28, 28))
    test = test.reshape((28, 28, 1))
#     test = np.array([test])
    data = np.array(test)
    data = data/255.0 # for range b/w 0-1
    data = data.reshape(1, 28, 28, 1).astype('float32')
#     plt.imshow(data)
    return data
    

def DrawContours(frame, points):
    for i in range(1, len(points)):
        if points[i - 1] is None or points[i] is None:
            continue
        cv2.line(frame, points[i-1], points[i], (0, 0, 255), 47)

drawingModule = mp.solutions.drawing_utils
handsModule = mp.solutions.hands
pTime = 0
digitlist = []
points = deque(maxlen=480)
detector = mp.solutions.hands
text=''

with handsModule.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.5, max_num_hands=2) as hands:    
    while (True):
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
         ### FPS frame rate
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        #cv2.putText(frame,f'FPS: {int(fps)}',(40,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
        cv2.putText(frame,'press c to clear and q to quit',(40,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
        #image = cv2.circle(frame, (cx,cy), 10, (0, 0, 255), -1)
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        imageHeight, imageWidth, _ = frame.shape
        
        # draw contours on the frames
        DrawContours(frame, points)
           
        # Draw and sample hand and Hand tracing    
        if results.multi_hand_landmarks != None:
            
            cv2.putText(frame, "Prediction is : " + str(text), (100,100), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255,0,0), 2,cv2.LINE_AA) 

            for handLandmarks in results.multi_hand_landmarks:
                #drawingModule.draw_landmarks(frame, handLandmarks, handsModule.HAND_CONNECTIONS)
                #get normalized position of index and thumb
                thumb, index, middle, ring, pinky = handLandmarks.landmark[4],handLandmarks.landmark[8], handLandmarks.landmark[12],handLandmarks.landmark[16],handLandmarks.landmark[20]
                index_k, middle_k, ring_k, pinky_k = handLandmarks.landmark[5],handLandmarks.landmark[9],handLandmarks.landmark[13],handLandmarks.landmark[17]
                
                # normalize landmark positions
                findex = drawingModule._normalized_to_pixel_coordinates(index.x, index.y, imageWidth, imageHeight)
                fthumb = drawingModule._normalized_to_pixel_coordinates(thumb.x, thumb.y, imageWidth, imageHeight)
#                 fmiddle = drawingModule._normalized_to_pixel_coordinates(middle.x, middle.y, imageWidth, imageHeight)
#                 fring = drawingModule._normalized_to_pixel_coordinates(ring.x, ring.y, imageWidth, imageHeight)
#                 fpinky = drawingModule._normalized_to_pixel_coordinates(pinky.x, pinky.y, imageWidth, imageHeight)
#                 findex_k = drawingModule._normalized_to_pixel_coordinates(index_k.x, index_k.y, imageWidth, imageHeight)
#                 fmiddle_k = drawingModule._normalized_to_pixel_coordinates(middle_k.x, middle_k.y, imageWidth, imageHeight)
#                 fring_k = drawingModule._normalized_to_pixel_coordinates(ring_k.x, ring_k.y, imageWidth, imageHeight)
#                 fpinky_k = drawingModule._normalized_to_pixel_coordinates(pinky_k.x, pinky_k.y, imageWidth, imageHeight)
                fmidpoint=((findex[0]+fthumb[0])//2, (findex[1]+fthumb[1])//2)
                
                ######### Circle Finger Tips ################
#                 cv2.circle(frame, findex, 15, (0, 255, 0), -1)
#                 cv2.circle(frame, fthumb, 15, (0, 255, 0), -1)
#                 cv2.circle(frame, fmiddle, 15, (0, 255, 0), -1)
#                 cv2.circle(frame, fring, 15, (0, 255, 0), -1)
#                 cv2.circle(frame, fpinky, 15, (0, 255, 0), -1)
                #############################################
                
                ######## Measure between index and thumb #############
                length = math.hypot(fthumb[0]-findex[0], fthumb[1]-findex[1])
                
                ########### Measure betwee other fingers #######################
                #length2 = math.hypot(findex[0]-fmiddle[0], findex[1]-fmiddle[1]) 
                #length3 = math.hypot(fmiddle[0]-fring[0], fmiddle[1]-fring[1]) 
                #length4 = math.hypot(fring[0]-fpinky[0], fring[1]-fpinky[1])
                
                #cv2.line(frame,findex,fthumb, (255,0,255),3) # draw a line between index and thumb
                cv2.circle(frame, fmidpoint, 15, (255, 0, 255), -1) # draw a circle between index and thumb
          
                
                   
                # when fingers together then below knuckles remove contours
                #if fmiddle[0]>fmiddle_k[0] and length2 <50:
                    

                #Press c to clear // buttons dont respond propery
                if cv2.waitKey(1) == ord('c'):
                    points.clear() 
                    
                # pinch index and thumb to draw    
                if length < 40:
                    points.append(fmidpoint)
                
                # write 
                #cv2.putText(frame, "Prediction is : " + str(text), (100,100), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255,0,0), 2,cv2.LINE_AA)                                     
        
        # red contour for frame 1
        red_mask = cv2.inRange(frame, red_low, red_high)
        red_mask = cv2.dilate(red_mask, None, iterations=1)
        red_mask = cv2.erode(red_mask, None, iterations=1)
        
        cv2.putText(frame, "Prediction is : " + str(text), (100,100), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255,0,0), 2,cv2.LINE_AA) 
               
        cv2.imshow("window_masked", red_mask)
        cv2.imshow('Test hand', frame)
        test_image = image_conversion(red_mask)
        text=np.argmax(model.predict(test_image))
        ### the buttons need to be spammed till respond
        if cv2.waitKey(1) == ord('q'):
            break # When everything is done, release the capture
        #if cv2.waitKey(1) == ord('z'):
            
        if cv2.waitKey(1) == ord('c'):
            points.clear()
            text = ''
        #elif cv2.waitKey(1) == ord('c'):q
            
    cv2.destroyAllWindows()
    #cap.release()

In [2]:
cap.release()